In [5]:
"""
============================================================
Micro-Bootstrap Ensemble (K=3) vs. Baseline MLP
============================================================
* Synthetic regression, 20 features
* Baseline: single Dropout MLP
* k3_boot: three shadow copies, per-batch bootstrap + variance penalty
* Metrics:  OOS RMSE  |  prediction-stability RMSE (√mean var across runs)
============================================================
"""
import copy, itertools, math, random, numpy as np, torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# -------------------------------------------------------------------
# 0.  Helpers
# -------------------------------------------------------------------
def set_seed(seed=0):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)

def pred_stability(pred_matrix: np.ndarray) -> float:
    """RMSE of predictions across bootstrap fits: √(mean var_per_sample)."""
    return math.sqrt(pred_matrix.var(axis=0).mean())

# -------------------------------------------------------------------
# 1.  Synthetic data
# -------------------------------------------------------------------
set_seed(0)
X, y = make_regression(
    n_samples=3500, n_features=20, n_informative=15, noise=20.0, random_state=0
)
X = StandardScaler().fit_transform(X).astype(np.float32)
y = y.astype(np.float32)
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.25, random_state=1
)
train_ds = TensorDataset(torch.tensor(X_tr), torch.tensor(y_tr))
TEST_X  = torch.tensor(X_te)
TEST_Y  = torch.tensor(y_te)

# -------------------------------------------------------------------
# 2.  Model
# -------------------------------------------------------------------
class DropMLP(nn.Module):
    def __init__(self, d_in=20, hid=64, p=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, 1)
        )
    def forward(self, x):  # → (B,)
        return self.net(x).squeeze(-1)

# -------------------------------------------------------------------
# 3.  Training routines
# -------------------------------------------------------------------
def train_baseline(seed, *, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    model = DropMLP()
    opt   = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_ds, batch_size=bs, sampler=RandomSampler(train_ds))

    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.mse_loss(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()

    # MC-Dropout average for evaluation
    model.eval()
    with torch.no_grad():
        preds = torch.stack([model(TEST_X) for _ in range(4)]).mean(0)
        rmse  = torch.sqrt(F.mse_loss(preds, TEST_Y)).item()
    return preds.numpy(), rmse


def train_k3(seed, *, K=3, lam=0.05, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    models = nn.ModuleList([DropMLP() for _ in range(K)])
    opt = torch.optim.Adam(
        itertools.chain(*[m.parameters() for m in models]), lr=lr
    )
    loader = DataLoader(train_ds, batch_size=bs, sampler=RandomSampler(train_ds))

    for _ in range(epochs):
        for xb, yb in loader:
            preds, sup_losses = [], []
            bootstrap_indices = []

            # ----- forward K bootstrap shadows -----
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                bootstrap_indices.append(idx)
                px = m(xb[idx])          # predictions
                preds.append(px)
                sup_losses.append(F.mse_loss(px, yb[idx]))

            preds     = torch.stack(preds)                 # (K,B)
            sup_loss  = torch.stack(sup_losses).mean()
            var_pen   = preds.var(dim=0, unbiased=False).mean()
            loss      = sup_loss + lam * var_pen

            # ----- update -----
            opt.zero_grad(); loss.backward(); opt.step()

    # ----- evaluation (ensemble average) -----
    for m in models: m.eval()
    with torch.no_grad():
        ens_preds = torch.stack([m(TEST_X) for m in models]).mean(0)
        rmse      = torch.sqrt(F.mse_loss(ens_preds, TEST_Y)).item()
    return ens_preds.numpy(), rmse

# -------------------------------------------------------------------
# 4.  Experiment: 6 fits each
# -------------------------------------------------------------------
R = 6                  # number of independent training replicates
baseline_preds, k3_preds = [], []
baseline_rmse,  k3_rmse  = [], []

for r in range(R):
    seed = 20250712 + r
    p0, e0 = train_baseline(seed)
    p1, e1 = train_k3(seed, K=3, lam=0.05)
    baseline_preds.append(p0); baseline_rmse.append(e0)
    k3_preds.append(p1);       k3_rmse.append(e1)

baseline_preds = np.stack(baseline_preds)
k3_preds       = np.stack(k3_preds)

print("=== Results over", R, "replicates ===")
print(f"Baseline  : RMSE = {np.mean(baseline_rmse):.2f} ± {np.std(baseline_rmse):.2f}   "
      f"Stability = {pred_stability(baseline_preds):.2f}")
print(f"k=3, λ=.05: RMSE = {np.mean(k3_rmse):.2f} ± {np.std(k3_rmse):.2f}   "
      f"Stability = {pred_stability(k3_preds):.2f}")

=== Results over 6 replicates ===
Baseline  : RMSE = 24.48 ± 0.21   Stability = 4.52
k=3, λ=.05: RMSE = 26.07 ± 0.40   Stability = 3.70


In [7]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import numpy as np, random, math, itertools, pandas as pd
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ----------------- Helpers -----------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def stability(pred_matrix: np.ndarray) -> float:
    """RMSE of predictions across fits."""
    return math.sqrt(pred_matrix.var(axis=0).mean())

# ----------------- Data -----------------
X, y = make_regression(
    n_samples=4000, n_features=20, n_informative=15, noise=20.0, random_state=0
)
X = StandardScaler().fit_transform(X).astype(np.float32)
y = y.astype(np.float32)
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=1)
train_ds = TensorDataset(torch.tensor(X_tr), torch.tensor(y_tr))
TEST_X = torch.tensor(X_te)
TEST_Y = torch.tensor(y_te)

# ----------------- Model -----------------
class DropMLP(nn.Module):
    def __init__(self, d_in=20, hid=64, p=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, 1)
        )
    def forward(self, x): return self.net(x).squeeze(-1)

# ----------------- Training routines -----------------
def train_baseline(seed, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    model = DropMLP()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_ds, batch_size=bs, sampler=RandomSampler(train_ds))
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.mse_loss(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        preds = torch.stack([model(TEST_X) for _ in range(4)]).mean(0)
        rmse = torch.sqrt(F.mse_loss(preds, TEST_Y)).item()
    return preds.numpy(), rmse

def train_k3(seed, lam=0.05, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    K = 3
    models = nn.ModuleList([DropMLP() for _ in range(K)])
    opt = torch.optim.Adam(itertools.chain(*[m.parameters() for m in models]), lr=lr)
    loader = DataLoader(train_ds, batch_size=bs, sampler=RandomSampler(train_ds))
    for _ in range(epochs):
        for xb, yb in loader:
            preds = []
            sup_losses = []
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                pred = m(xb[idx])
                preds.append(pred)
                sup_losses.append(F.mse_loss(pred, yb[idx]))
            preds = torch.stack(preds)  # K x B
            sup_loss = torch.stack(sup_losses).mean()
            var_pen = preds.var(dim=0, unbiased=False).mean()
            loss = sup_loss + lam * var_pen
            opt.zero_grad(); loss.backward(); opt.step()
    for m in models:
        m.eval()
    with torch.no_grad():
        ensemble_preds = torch.stack([m(TEST_X) for m in models]).mean(0)
        rmse = torch.sqrt(F.mse_loss(ensemble_preds, TEST_Y)).item()
    return ensemble_preds.numpy(), rmse

# ----------------- Comprehensive Evaluation -----------------
R = 30  # number of independent replicates
baseline_preds_list, k3_preds_list = [], []
baseline_rmse_list, k3_rmse_list = [], []

for r in range(R):
    seed = 12345 + r
    bpred, brmse = train_baseline(seed)
    kpred, krmse = train_k3(seed)
    baseline_preds_list.append(bpred); baseline_rmse_list.append(brmse)
    k3_preds_list.append(kpred);        k3_rmse_list.append(krmse)

baseline_preds = np.stack(baseline_preds_list)
k3_preds = np.stack(k3_preds_list)

summary = pd.DataFrame({
    "method": ["baseline", "k=3"],
    "avg_test_RMSE": [np.mean(baseline_rmse_list), np.mean(k3_rmse_list)],
    "std_test_RMSE": [np.std(baseline_rmse_list), np.std(k3_rmse_list)],
    "prediction_stability_RMSE": [stability(baseline_preds), stability(k3_preds)]
})

summary

,method,avg_test_RMSE,std_test_RMSE,prediction_stability_RMSE
0,baseline,24.003791,0.391080,5.367281
1,k=3,27.379538,0.501202,3.457137


In [15]:
#!/usr/bin/env python
# -------------------------------------------------------------
# Bootstrap‑variance experiment: synthetic regression + Adult income
# -------------------------------------------------------------
import math, random, itertools, inspect, warnings
import numpy as np, pandas as pd, torch
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.datasets import make_regression, fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from packaging import version

# ---------------- helpers ----------------
def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)

def stability(pred_mat: np.ndarray) -> float:
    """sqrt(mean variance) across fits (rows) at each sample (cols)"""
    return math.sqrt(pred_mat.var(axis=0).mean())

# ---------------- synthetic regression data ----------------
set_seed(0)
X_syn, y_syn = make_regression(
    n_samples=4000, n_features=20, n_informative=15, noise=20.0, random_state=0
)
X_syn = StandardScaler().fit_transform(X_syn).astype("float32")
y_syn = y_syn.astype("float32")
Xtr_syn, Xte_syn, ytr_syn, yte_syn = train_test_split(
    X_syn, y_syn, test_size=0.25, random_state=1
)
train_syn = TensorDataset(torch.tensor(Xtr_syn), torch.tensor(ytr_syn))
TEST_X_syn = torch.tensor(Xte_syn)
TEST_Y_syn = torch.tensor(yte_syn)

# ---------------- Adult income data ----------------
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

adult = fetch_openml("adult", version=2, as_frame=True)
X_df = adult.data
y_ser = (adult.target == ">50K").astype("int64")          # 0/1 label

num_cols = X_df.select_dtypes("number").columns.to_list()
cat_cols = X_df.select_dtypes("object").columns.to_list()

# robust OneHotEncoder
if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
else:
    enc = OneHotEncoder(handle_unknown="ignore", sparse=False)

preproc = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", enc,            cat_cols)
])
X_adult = preproc.fit_transform(X_df).astype("float32")
y_adult = y_ser.to_numpy()

Xtr_A, Xte_A, ytr_A, yte_A = train_test_split(
    X_adult, y_adult, test_size=0.25, random_state=1, stratify=y_adult
)
train_A = TensorDataset(torch.tensor(Xtr_A), torch.tensor(ytr_A))
TEST_X_A = torch.tensor(Xte_A)
TEST_Y_A = torch.tensor(yte_A)

# ---------------- model defs ----------------
class DropMLP(nn.Module):
    def __init__(self, d_in, hid=64, p=0.2, out_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, hid), nn.ReLU(), nn.Dropout(p),
            nn.Linear(hid, out_dim)
        )
    def forward(self, x):
        out = self.net(x)
        return out.squeeze(-1) if out.shape[1] == 1 else out

# ---------------- training functions ----------------
def train_syn_baseline(seed, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    model = DropMLP(d_in=X_syn.shape[1], out_dim=1)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_syn, batch_size=bs, sampler=RandomSampler(train_syn))
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.mse_loss(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        preds = torch.stack([model(TEST_X_syn) for _ in range(4)]).mean(0)
        rmse  = torch.sqrt(F.mse_loss(preds, TEST_Y_syn)).item()
    return preds.numpy(), rmse

def train_syn_k3(seed, lam=0.05, epochs=25, bs=64, lr=1e-3):
    set_seed(seed)
    K = 3
    models = nn.ModuleList([DropMLP(d_in=X_syn.shape[1], out_dim=1) for _ in range(K)])
    opt = torch.optim.Adam(itertools.chain(*(m.parameters() for m in models)), lr=lr)
    loader = DataLoader(train_syn, batch_size=bs, sampler=RandomSampler(train_syn))
    for _ in range(epochs):
        for xb, yb in loader:
            preds, sup = [], []
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                p   = m(xb[idx])
                preds.append(p)
                sup.append(F.mse_loss(p, yb[idx]))
            preds = torch.stack(preds)              # (K,B)
            loss  = torch.stack(sup).mean() + lam * preds.var(0).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    for m in models: m.eval()
    with torch.no_grad():
        ens = torch.stack([m(TEST_X_syn) for m in models]).mean(0)
        rmse = torch.sqrt(F.mse_loss(ens, TEST_Y_syn)).item()
    return ens.numpy(), rmse

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

def train_A_baseline(seed, epochs=20, bs=256, lr=3e-4):
    set_seed(seed)
    d_in = X_adult.shape[1]
    model = DropMLP(d_in=d_in, hid=128, p=0.3, out_dim=2)
    opt   = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_A, batch_size=bs, sampler=RandomSampler(train_A))
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.cross_entropy(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        logits = torch.stack([model(TEST_X_A) for _ in range(4)]).mean(0)
        acc    = accuracy(logits, TEST_Y_A)
    return logits.numpy(), acc

def train_A_k3(seed, lam=0.05, epochs=20, bs=256, lr=3e-4):
    set_seed(seed)
    d_in = X_adult.shape[1]
    K = 3
    models = nn.ModuleList([DropMLP(d_in=d_in, hid=128, p=0.3, out_dim=2) for _ in range(K)])
    opt = torch.optim.Adam(itertools.chain(*(m.parameters() for m in models)), lr=lr)
    loader = DataLoader(train_A, batch_size=bs, sampler=RandomSampler(train_A))
    for _ in range(epochs):
        for xb, yb in loader:
            logitsK, sup = [], []
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                out = m(xb[idx])
                logitsK.append(out)
                sup.append(F.cross_entropy(out, yb[idx]))
            logitsK = torch.stack(logitsK)          # (K,B,2)
            loss = torch.stack(sup).mean() + lam * logitsK.var(0).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    for m in models: m.eval()
    with torch.no_grad():
        logits = torch.stack([m(TEST_X_A) for m in models]).mean(0)
        acc    = accuracy(logits, TEST_Y_A)
    return logits.numpy(), acc

# ---------------- run 30 replicates ----------------
R = 30
syn_base_preds, syn_k3_preds = [], []
syn_base_rmse, syn_k3_rmse   = [], []
A_base_logits, A_k3_logits   = [], []
A_base_acc,   A_k3_acc       = [], []

for r in range(R):
    seed = 1234 + r
    bp, br = train_syn_baseline(seed)
    kp, kr = train_syn_k3(seed)
    syn_base_preds.append(bp); syn_base_rmse.append(br)
    syn_k3_preds.append(kp);   syn_k3_rmse.append(kr)

    bl, ba = train_A_baseline(seed)
    kl, ka = train_A_k3(seed)
    A_base_logits.append(bl); A_base_acc.append(ba)
    A_k3_logits.append(kl);   A_k3_acc.append(ka)

# ---------------- print summaries ----------------
syn_summary = pd.DataFrame({
    "method": ["baseline", "k=3"],
    "avg_RMSE": [np.mean(syn_base_rmse), np.mean(syn_k3_rmse)],
    "std_RMSE": [np.std(syn_base_rmse), np.std(syn_k3_rmse)],
    "prediction_stability_RMSE": [
        stability(np.stack(syn_base_preds)),
        stability(np.stack(syn_k3_preds))
    ]
})

adult_summary = pd.DataFrame({
    "method": ["baseline", "k=3"],
    "avg_Accuracy": [np.mean(A_base_acc), np.mean(A_k3_acc)],
    "std_Accuracy": [np.std(A_base_acc), np.std(A_k3_acc)],
    "logit_stability_RMSE": [
        stability(np.stack(A_base_logits)),
        stability(np.stack(A_k3_logits))
    ]
})

print("\n=== Synthetic regression (30 runs) ===")
print(syn_summary.round(3))
print("\n=== Adult income (30 runs) ===")
print(adult_summary.round(3))


=== Synthetic regression (30 runs) ===
     method  avg_RMSE  std_RMSE  prediction_stability_RMSE
0  baseline    23.878     0.371                      5.293
1       k=3    29.457     0.642                      3.275

=== Adult income (30 runs) ===
     method  avg_Accuracy  std_Accuracy  logit_stability_RMSE
0  baseline         0.826         0.001                 0.236
1       k=3         0.825         0.001                 0.044


In [16]:
# ------------------------------------------------------------
# California Housing (regression)
# ------------------------------------------------------------
from sklearn.datasets import fetch_california_housing

cal = fetch_california_housing(as_frame=True)
X_cal = cal.data.values.astype("float32")
y_cal = cal.target.values.astype("float32")

from sklearn.preprocessing import StandardScaler
X_cal = StandardScaler().fit_transform(X_cal).astype("float32")

Xtr_C, Xte_C, ytr_C, yte_C = train_test_split(
    X_cal, y_cal, test_size=0.25, random_state=1
)
train_C = TensorDataset(torch.tensor(Xtr_C), torch.tensor(ytr_C))
TEST_X_C = torch.tensor(Xte_C)
TEST_Y_C = torch.tensor(yte_C)

# ------------------------------------------------------------
# Credit Card Default (binary classification)
# ------------------------------------------------------------
credit = fetch_openml("credit-g", version=1, as_frame=True)     # German credit
X_cr = credit.data
y_cr = (credit.target == "good").astype("int64")      # 1 = good credit

num_cols_cr = X_cr.select_dtypes("number").columns.to_list()
cat_cols_cr = X_cr.select_dtypes("object").columns.to_list()

# robust OneHotEncoder (same trick as before)
if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    enc_cr = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
else:
    enc_cr = OneHotEncoder(handle_unknown="ignore", sparse=False)

pre_cr = ColumnTransformer([
    ("num", StandardScaler(), num_cols_cr),
    ("cat", enc_cr,          cat_cols_cr)
])
X_cr = pre_cr.fit_transform(X_cr).astype("float32")
y_cr = y_cr.to_numpy()

Xtr_CR, Xte_CR, ytr_CR, yte_CR = train_test_split(
    X_cr, y_cr, test_size=0.25, random_state=1, stratify=y_cr
)
train_CR = TensorDataset(torch.tensor(Xtr_CR), torch.tensor(ytr_CR))
TEST_X_CR = torch.tensor(Xte_CR)
TEST_Y_CR = torch.tensor(yte_CR)

/Users/soodoku/opt/anaconda3/envs/py311ds/lib/python3.11/site-packages/sklearn/datasets/_openml.py:323: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1. Available versions:
- version 1, status: active
  url: https://www.openml.org/search?type=data&id=31
- version 2, status: active
  url: https://www.openml.org/search?type=data&id=44096

  warn(warning_msg)


In [17]:
# ---------------- California Housing training ----------------
def train_C_baseline(seed, epochs=20, bs=128, lr=1e-3):
    set_seed(seed)
    model = DropMLP(d_in=X_cal.shape[1], hid=64, p=0.2, out_dim=1)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_C, batch_size=bs, sampler=RandomSampler(train_C))
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.mse_loss(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        preds = torch.stack([model(TEST_X_C) for _ in range(4)]).mean(0)
        rmse  = torch.sqrt(F.mse_loss(preds, TEST_Y_C)).item()
    return preds.numpy(), rmse

def train_C_k3(seed, lam=0.05, epochs=20, bs=128, lr=1e-3):
    set_seed(seed)
    K = 3
    models = nn.ModuleList([DropMLP(d_in=X_cal.shape[1], hid=64, p=0.2, out_dim=1) for _ in range(K)])
    opt = torch.optim.Adam(itertools.chain(*(m.parameters() for m in models)), lr=lr)
    loader = DataLoader(train_C, batch_size=bs, sampler=RandomSampler(train_C))
    for _ in range(epochs):
        for xb, yb in loader:
            preds, sup = [], []
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                p = m(xb[idx]); preds.append(p)
                sup.append(F.mse_loss(p, yb[idx]))
            preds = torch.stack(preds)
            loss  = torch.stack(sup).mean() + lam * preds.var(0).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    for m in models: m.eval()
    with torch.no_grad():
        ens = torch.stack([m(TEST_X_C) for m in models]).mean(0)
        rmse = torch.sqrt(F.mse_loss(ens, TEST_Y_C)).item()
    return ens.numpy(), rmse

# ---------------- Credit Default training ----------------
def train_CR_baseline(seed, epochs=20, bs=256, lr=3e-4):
    set_seed(seed)
    d_in = X_cr.shape[1]
    model = DropMLP(d_in=d_in, hid=128, p=0.3, out_dim=2)
    opt   = torch.optim.Adam(model.parameters(), lr=lr)
    loader = DataLoader(train_CR, batch_size=bs, sampler=RandomSampler(train_CR))
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            loss = F.cross_entropy(model(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        logits = torch.stack([model(TEST_X_CR) for _ in range(4)]).mean(0)
        acc    = accuracy(logits, TEST_Y_CR)
    return logits.numpy(), acc

def train_CR_k3(seed, lam=0.05, epochs=20, bs=256, lr=3e-4):
    set_seed(seed)
    d_in = X_cr.shape[1]; K = 3
    models = nn.ModuleList([DropMLP(d_in=d_in, hid=128, p=0.3, out_dim=2) for _ in range(K)])
    opt = torch.optim.Adam(itertools.chain(*(m.parameters() for m in models)), lr=lr)
    loader = DataLoader(train_CR, batch_size=bs, sampler=RandomSampler(train_CR))
    for _ in range(epochs):
        for xb, yb in loader:
            logitsK, sup = [], []
            for m in models:
                idx = torch.randint(0, len(xb), (len(xb),))
                out = m(xb[idx])
                logitsK.append(out)
                sup.append(F.cross_entropy(out, yb[idx]))
            logitsK = torch.stack(logitsK)
            loss = torch.stack(sup).mean() + lam * logitsK.var(0).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    for m in models: m.eval()
    with torch.no_grad():
        logits = torch.stack([m(TEST_X_CR) for m in models]).mean(0)
        acc    = accuracy(logits, TEST_Y_CR)
    return logits.numpy(), acc

In [18]:
# ---- arrays to collect
cal_base_preds, cal_k3_preds = [], []
cal_base_rmse, cal_k3_rmse   = [], []
CR_base_logits, CR_k3_logits = [], []
CR_base_acc,   CR_k3_acc     = [], []

for r in range(R):
    seed = 1234 + r
    # existing synthetic + adult code ...
    # ---- California
    bp, br = train_C_baseline(seed)
    kp, kr = train_C_k3(seed)
    cal_base_preds.append(bp); cal_base_rmse.append(br)
    cal_k3_preds.append(kp);   cal_k3_rmse.append(kr)
    # ---- Credit Default
    bl, ba = train_CR_baseline(seed)
    kl, ka = train_CR_k3(seed)
    CR_base_logits.append(bl); CR_base_acc.append(ba)
    CR_k3_logits.append(kl);   CR_k3_acc.append(ka)

# ---- summaries
cal_summary = pd.DataFrame({
    "method": ["baseline","k=3"],
    "avg_RMSE":[np.mean(cal_base_rmse), np.mean(cal_k3_rmse)],
    "std_RMSE":[np.std(cal_base_rmse), np.std(cal_k3_rmse)],
    "prediction_stability_RMSE":[stability(np.stack(cal_base_preds)),
                                 stability(np.stack(cal_k3_preds))]
})

credit_summary = pd.DataFrame({
    "method": ["baseline","k=3"],
    "avg_Accuracy":[np.mean(CR_base_acc), np.mean(CR_k3_acc)],
    "std_Accuracy":[np.std(CR_base_acc), np.std(CR_k3_acc)],
    "logit_stability_RMSE":[stability(np.stack(CR_base_logits)),
                            stability(np.stack(CR_k3_logits))]
})

print("\n=== California Housing (30 runs) ===")
print(cal_summary.round(3))
print("\n=== Credit Default (30 runs) ===")
print(credit_summary.round(3))


=== California Housing (30 runs) ===
     method  avg_RMSE  std_RMSE  prediction_stability_RMSE
0  baseline     0.591     0.005                      0.074
1       k=3     0.598     0.004                      0.054

=== Credit Default (30 runs) ===
     method  avg_Accuracy  std_Accuracy  logit_stability_RMSE
0  baseline         0.697         0.009                 0.118
1       k=3         0.688         0.006                 0.061
